In [ ]:
import os
import requests
import json
import pandas as pd
import re
import copy
import time
import random
from typing import List
from objectss import movie, comment


In [ ]:
headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55',
    'cookie': 'douban-fav-remind=1; gr_user_id=430d4823-0bb8-4329-adba-4333db071744; _ga=GA1.1.1595394736.1611848456; _ga_RXNMP372GL=GS1.1.1637919254.1.1.1637921082.0; viewed="26340138_1822780_21371175_30276702_3393056_26901177_26110579_30465903"; bid=5q98-_Us4Sw; __gads=ID=a50462eb29c4d08e-2299ccaa64d100ff:T=1648378880:RT=1648378880:S=ALNI_MaFvH7_ymas_6_imHe2djV094YY4A; ll="118378"; __gpi=UID=000004a04f2f7458:T=1649398885:RT=1649398885:S=ALNI_MYENVr1Ij1B6mPh6eTj0Hf0LaLVtg; __utmz=30149280.1649405301.22.13.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utma=30149280.1595394736.1611848456.1649405301.1649424095.23; __utmc=30149280; __utmt=1; __utmb=30149280.1.10.1649424095; ap_v=0,6.0'}

In [ ]:
MOVIE_SEARCH_BASE_PATH = 'https://movie.douban.com/j/search_subjects'

def generate_movie_get_parameters(page_start: int = 0, page_limit: int = 220, sort: str = 'recommend', tag: str = '热门', type: str = 'movie'):
    return {
        'type': type,
        'tag': tag,
        'sort': sort,
        'page_limit': page_limit,
        'page_start': page_start
    }

In [ ]:
res = requests.get(MOVIE_SEARCH_BASE_PATH, params=generate_movie_get_parameters(), headers=headers)

In [ ]:
a=json.loads(res.text)

In [ ]:
iplist=[]
with open('../data/ipp.txt')as f:
    iplist=f.readlines()

def getip():
    proxy=iplist[random.randint(0,len(iplist)-1)]
    proxy=proxy.replace('\n','')
    proxies={'http':'http://'+str(proxy)}
    return proxies

def get_movie_comment_base_path(id: int) -> str:
    return f'https://movie.douban.com/subject/{id}/comments?start=0&limit=200&status=P&sort=new_score'

def generate_movie_get_parameters(start: int = 0, limit: int = 20, sort: str = 'new_score', status: str = 'P'):
    return {
        start: start,
        limit: limit,
        sort: sort,
        status: status
    }

In [ ]:
#response=requests.get(base_url,headers=headers)
#
#html=response.text
## res 解析需要 beautifulsoup4


In [ ]:
movie0 = [0]*3
Movie0 = []
comment0 = [0] * 5
Comment0 = []

def download_movies() -> List[movie.Movie]:
    for i in range(220):
        mov_id = a["subjects"][i]["id"]
        mov_title = a["subjects"][i]["title"]
        mov_rate = a["subjects"][i]["rate"]

        movie0[0] = mov_id
        movie0[1] = mov_title
        movie0[2] = mov_rate
        Movie0.append(copy.deepcopy(movie0))
    return Movie0

In [ ]:
DATA_BASE_DIR = os.path.join('..', 'data')
# maybe consider csv
DATA_TXT_PATH = os.path.join(DATA_BASE_DIR, 'movie.csv')
DATA_TXT_PATH2 = os.path.join(DATA_BASE_DIR, 'comment.csv')
if not os.path.exists(DATA_BASE_DIR):
    os.mkdir(DATA_BASE_DIR)

In [ ]:
mov0=download_movies()
def download_comments() -> List[comment.ShortComment]:
    for i in range(220):
        time.sleep(random.randint(3,5))
        res = requests.get(get_movie_comment_base_path(a["subjects"][i]["id"]),
                           params=generate_movie_get_parameters(), proxies=getip(), headers=headers)
        res.encoding = 'utf-8'
        html = res.text

        com_movie_id = a["subjects"][i]["id"]
        com_user_id = re.compile('class="">(.*?)</a>')
        url_user_id = com_user_id.findall(html)
        com_rate = re.compile('title="(.*?)"></span>')
        url_rate = com_rate.findall(html)
        com_time = re.compile('<span class="comment-time " title="(.*?)">')
        url_time = com_time.findall(html)
        com_content = re.compile('<span class="short">(.*?)</span>')
        url_content = com_content.findall(html)

        print(i)
        c=len(url_content)
        if c>105:
            for k in range(105):
                comment0[0]=com_movie_id
                comment0[1]=url_user_id[k]

                if url_rate[k] == '力荐':
                    comment0[2] = 5
                elif url_rate[k] == '推荐':
                    comment0[2] = 4
                elif url_rate[k] == '还行':
                    comment0[2] = 3
                elif url_rate[k] == '较差':
                    comment0[2] = 2
                else:
                    comment0[2] = 1
                comment0[3] = url_time[k]
                url_content[k] = url_content[k].replace(',',' ')
                comment0[4] = url_content[k]
                Comment0.append(copy.deepcopy(comment0))
        else:
            mov0.pop(i)

    return Comment0

In [ ]:
Com=[]
for i in download_comments():
    Com.append(copy.deepcopy(i))
dataframe = pd.DataFrame(Com)
dataframe.to_csv(DATA_TXT_PATH2,index = False,header=['movie_id','user_id','rate','time','content'])

In [ ]:
Mov=[]
for i in download_movies():
    Mov.append(copy.deepcopy(i))
dataframe = pd.DataFrame(Mov)
dataframe.to_csv(DATA_TXT_PATH,index = False,header=['id','title','rate'])

In [ ]:
if __name__ == '__main__':
    print(DATA_BASE_DIR)